# SPLADE for Portuguese

Authors: Leonardo Ávila, Monique Monteiro

Inspired by https://github.com/naver/splade

This notebook contains the indexing code to build a SPLADEv2 invered index from a trained checkpoint.  Dataset: mRobust.

In [ ]:
!nvidia-smi

Thu Jun 29 01:39:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
user = "monique" # monique
if user == "monique":
    main_dir = '/content/gdrive/MyDrive/Unicamp-projeto-final/'
else:
    main_dir = '/content/gdrive/MyDrive/Unicamp/IA368-DD/'

## Libraries installation

In [ ]:
%%shell
pip install pytrec_eval
pip install git+https://github.com/naver/splade.git -q
pip install hydra-core --upgrade

  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=293482 sha256=1b66ad9755d96a5badcc55416b26ca210c4af40456eaf68f5c9e1ee0fddced6a
  Stored in directory: /root/.cache/pip/wheels/51/3a/cd/dcc1ddfc763987d5cb237165d8ac249aa98a23ab90f67317a8
Successfully built pytrec_eval
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 62.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 111.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
import pytrec_eval

## Copying data files to the expected data structure

In [ ]:
!git clone https://github.com/naver/splade.git

Cloning into 'splade'...
remote: Enumerating objects: 703, done.
remote: Counting objects: 100% (236/236), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 703 (delta 167), reused 144 (delta 138), pack-reused 467
Receiving objects: 100% (703/703), 3.14 MiB | 974.00 KiB/s, done.
Resolving deltas: 100% (379/379), done.


In [ ]:
import shutil
import os

shutil.copyfile(f"{main_dir}Projeto Final/data.zip", "/content/data.zip")
!unzip data.zip

Archive:  data.zip
   creating: content/splade_data/
   creating: content/splade_data/.ipynb_checkpoints/
   creating: content/splade_data/m_robust/
   creating: content/splade_data/m_robust/.ipynb_checkpoints/
  inflating: content/splade_data/m_robust/queries.tsv  
  inflating: content/splade_data/m_robust/qrels.robust04.txt  
  inflating: content/splade_data/m_robust/corpus.tsv  
   creating: content/splade_data/m_marco/
  inflating: content/splade_data/m_marco/queries_train.tsv  
  inflating: content/splade_data/m_marco/queries_dev.tsv  
  inflating: content/splade_data/m_marco/corpus.tsv  
  inflating: content/splade_data/m_marco/queries_dev.full.tsv  


In [ ]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/config_splade_pt.yaml" splade/conf/

In [ ]:
#!mkdir -p splade/data/pt/triplets
!cp content/data/m_marco/triplets.tsv splade/data/pt/triplets/raw.tsv

# LEONARDO: ALTEREI AQUI PRA PEGAR UM ARQUIVO MENOR DE TRIPLETS
!mkdir -p splade/data/pt/triplets
#!head -n 1000 content/data/m_marco/triplets.tsv > splade/data/pt/triplets/raw.tsv

cp: cannot stat 'content/data/m_marco/triplets.tsv': No such file or directory


In [ ]:
!wc -l splade/data/pt/triplets/raw.tsv

wc: splade/data/pt/triplets/raw.tsv: No such file or directory


In [ ]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/train/data/pt.yaml" splade/conf/train/data

### Generating/copying validation collection

Now we need to generate a validation collection: it will be based on the same validation data used in https://github.com/naver/splade.

In [ ]:
import pandas as pd

In [ ]:
!mkdir -p "content/data/m_marco/val_retrieval/collection"

In [ ]:
shutil.copyfile(f"{main_dir}Projeto Final/data_m_marco_val_retrieval.zip", "/content/data_m_marco_val_retrieval.zip")
!unzip data_m_marco_val_retrieval.zip

Archive:  data_m_marco_val_retrieval.zip
   creating: content/data/m_marco/val_retrieval/queries/
  inflating: content/data/m_marco/val_retrieval/queries/raw.tsv  
  inflating: content/data/m_marco/val_retrieval/collection/raw.tsv  


In [ ]:
import csv
import tqdm

#Lookups the respective translated passages.
def gen_val_collection():
  df_val_collection = pd.read_csv(f"{main_dir}Projeto Final/msmarco/val_retrieval/collection/raw.tsv", sep='\t', header=None)
  df_corpus = pd.read_csv("content/data/m_marco/corpus.tsv", sep='\t', header=None)

  with open("content/data/m_marco/val_retrieval/collection/raw.tsv", 'w') as f:
    writer = csv.writer(f, delimiter='\t')  # Set the delimiter as a tab

    for index, row in tqdm.tqdm(df_val_collection.iterrows(), total=df_val_collection.shape[0]):
      doc_id = row[0]
      doc_df = df_corpus[df_corpus[0] == doc_id]
      if doc_df.shape[0] >= 1:
        doc_text = doc_df.iloc[0,1]
        writer.writerow([doc_id, doc_text])

In [ ]:
!mkdir -p splade/data/pt/val_retrieval/collection
!cp content/data/m_marco/val_retrieval/collection/raw.tsv splade/data/pt/val_retrieval/collection

### Generating/copying validation queries

In [ ]:
!mkdir -p "content/data/m_marco/val_retrieval/queries"

In [ ]:
import csv
import tqdm

def gen_val_queries():
  df_val_queries = pd.read_csv(f"{main_dir}Projeto Final/msmarco/val_retrieval/queries/raw.tsv", sep='\t', header=None)
  df_queries_dev_full = pd.read_csv("content/data/m_marco/queries_dev.full.tsv", sep='\t', header=None)

  with open("content/data/m_marco/val_retrieval/queries/raw.tsv", 'w') as f:
    writer = csv.writer(f, delimiter='\t')  # Set the delimiter as a tab

    for index, row in tqdm.tqdm(df_val_queries.iterrows(), total=df_val_queries.shape[0]):
      q_id = row[0]
      q_df = df_queries_dev_full[df_queries_dev_full[0] == q_id]
      if q_df.shape[0] >= 1:
        q_text = q_df.iloc[0,1]
        writer.writerow([q_id, q_text])

In [ ]:
!mkdir -p splade/data/pt/val_retrieval/queries
!cp content/data/m_marco/val_retrieval/queries/raw.tsv splade/data/pt/val_retrieval/queries

### Copying QREL validation data

In [ ]:
!cp "{main_dir}Projeto Final/msmarco/val_retrieval/qrel.json" splade/data/pt/val_retrieval

### Copying full collection

For indexing we'll validate the model on mROBUST dataset.  So, its corpus should be used as full collection.

In [ ]:
!mkdir -p splade/data/pt/full_collection

Preprocesses mROBUST dataset in order to replace alphanumerical IDs with integer ones.

In [ ]:
import pandas as pd

#df_mrobust_corpus = pd.read_csv("content/data/m_robust/corpus.tsv", sep='\t', header=None)
#df_mrobust_corpus.head()
map_id_to_text = dict()

#with open("content/data/m_robust/corpus.tsv", 'r') as file:
with open("content/splade_data/m_robust/corpus.tsv", 'r') as file:
  for line in file:
    fields = line.split('\t')
    id = fields[0]
    text = fields[1]
    map_id_to_text[id] = text

In [ ]:
!head -n 10 content/data/m_robust/corpus.tsv

head: cannot open 'content/data/m_robust/corpus.tsv' for reading: No such file or directory


In [ ]:
doc_ids = map_id_to_text.keys()

In [ ]:
map_id_to_int = {value:index for index, value in enumerate(doc_ids)}

In [ ]:
new_map_id_to_text = {map_id_to_int[id]: text for id, text in map_id_to_text.items()}

In [ ]:
len(new_map_id_to_text)

528032

In [ ]:
with open('new_corpus.tsv', 'w') as file:
  for id, text in new_map_id_to_text.items():
    file.write(f"{id}\t{text}")

In [ ]:
!head content/data/m_robust/corpus.tsv

head: cannot open 'content/data/m_robust/corpus.tsv' for reading: No such file or directory


In [ ]:
!head new_corpus.tsv

0	930105 FT 05 JAN 93 / Redland concorda acordos europeus de 59 milhões de libras REDLAND, o grupo de materiais de construção que pagou 624 milhões de libras em ações pela Steetley no ano passado, arrecadou 100 milhões de libras por meio de alienações. Está reinvestindo 58,6 milhões de libras em suas atividades de azulejos e tijolos na Europa continental. A Redland está formando uma joint venture com a Koramic, uma empresa privada belga, que engloba as fábricas de tijolos das duas empresas na Holanda, Alemanha e Bélgica. A nova empresa será a maior fabricante de tijolos de fachada da Europa continental. Os lucros das atividades combinadas em 1992 teriam sido de 15,9 milhões de libras, dos quais a parte de Redland teria sido de 5,8 milhões de libras. A joint venture 50:50 deterá 70 por cento das atividades combinadas, dando a Redland uma participação econômica de 35 por cento, mas controle conjunto. Redland também receberá 17,3 milhões de libras em dinheiro. A Redland está aumentando su

In [ ]:
!wc -l content/data/m_robust/corpus.tsv

wc: content/data/m_robust/corpus.tsv: No such file or directory


In [ ]:
!wc -l new_corpus.tsv

528032 new_corpus.tsv


In [ ]:
!cp new_corpus.tsv splade/data/pt/full_collection/raw.tsv

In [ ]:
!head splade/data/pt/full_collection/raw.tsv

0	930105 FT 05 JAN 93 / Redland concorda acordos europeus de 59 milhões de libras REDLAND, o grupo de materiais de construção que pagou 624 milhões de libras em ações pela Steetley no ano passado, arrecadou 100 milhões de libras por meio de alienações. Está reinvestindo 58,6 milhões de libras em suas atividades de azulejos e tijolos na Europa continental. A Redland está formando uma joint venture com a Koramic, uma empresa privada belga, que engloba as fábricas de tijolos das duas empresas na Holanda, Alemanha e Bélgica. A nova empresa será a maior fabricante de tijolos de fachada da Europa continental. Os lucros das atividades combinadas em 1992 teriam sido de 15,9 milhões de libras, dos quais a parte de Redland teria sido de 5,8 milhões de libras. A joint venture 50:50 deterá 70 por cento das atividades combinadas, dando a Redland uma participação econômica de 35 por cento, mas controle conjunto. Redland também receberá 17,3 milhões de libras em dinheiro. A Redland está aumentando su

In [ ]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/index/pt.yaml" splade/conf/index

### Retrieval and evaluation configuration

Here, we'll configure retrieval and evaluation for mROBUST dataset.

In [ ]:
!mkdir -p splade/data/pt/dev_queries

In [ ]:
!cp content/data/m_robust/queries.tsv splade/data/pt/dev_queries/raw.tsv

cp: cannot stat 'content/data/m_robust/queries.tsv': No such file or directory


Converts QREL information to JSON fomat.

In [ ]:
#TODO: Devo fazer o dump apenas dos que têm relevância 1?
from collections import defaultdict
import json

qrel = defaultdict(dict)

#with open("content/data/m_robust/qrels.robust04.txt", 'r') as file:
with open("content/splade_data/m_robust/qrels.robust04.txt", 'r') as file:
  for line in file:
    fields = line.split()
    q_id = fields[0]
    if fields[2] in map_id_to_int:
      doc_id = map_id_to_int[fields[2]]
      rel = fields[3]
      qrel[q_id][doc_id] = rel

with open('splade/data/pt/dev_qrel.json', 'w') as file:
    json.dump(qrel, file)

In [ ]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/retrieve_evaluate/pt.yaml" splade/conf/retrieve_evaluate

### Flops estimation

We'll use the same test set (mRobust) to estimate flops.

In [ ]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/flops/pt.yaml" splade/conf/flops

## Indexing and retrieving

In [ ]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/train/model/splade_bertimbau_base.yaml" splade/conf/train/model/

In [ ]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/config_splade_pt.yaml" splade/conf/
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/train/config/splade_pt.yaml" splade/conf/train/config

In [ ]:
shutil.copyfile(f"{main_dir}Projeto Final/experiments.zip", "/content/experiments.zip")

'/content/experiments.zip'

In [ ]:
!unzip /content/experiments_ids_384_max.zip
!mv /content/content/splade/experiments /content/experiments

In [ ]:
!cp -r experiments/pt splade/experiments/

In [ ]:
!ls experiments/pt/checkpoint/

config.yaml  model_ckpt   training_perf.txt  validation_perf.txt
model	     tensorboard  val_full_ranking


In [ ]:
!mkdir -p splade/experiments/pt
!mv splade/experiments/checkpoint splade/experiments/pt

In [ ]:
%%shell
cd splade/
export PYTHONPATH=$PYTHONPATH:$(pwd)
export SPLADE_CONFIG_NAME="config_splade_pt.yaml"
python3 -m splade.index \
  config.checkpoint_dir=experiments/pt/checkpoint \
  config.index_dir=experiments/pt/index \
  config.out_dir=experiments/pt/out

In [ ]:
!zip -r index_ids_384_max.zip splade/experiments/pt/index

  adding: splade/experiments/pt/index/ (stored 0%)
  adding: splade/experiments/pt/index/doc_ids.pkl (deflated 66%)
  adding: splade/experiments/pt/index/index_stats.json (stored 0%)
  adding: splade/experiments/pt/index/array_index.h5py (deflated 36%)
  adding: splade/experiments/pt/index/index_dist.json (deflated 64%)


In [ ]:
shutil.copyfile("index_ids_384_max.zip", f"{main_dir}Projeto Final/index_ids_384_max.zip",)

'/content/gdrive/MyDrive/Unicamp/IA368-DD/Projeto Final/index_ids_384_max.zip'